In [2]:
import pandas as pd
import seaborn as sns

In [3]:
patient_df = pd.read_csv('/Users/josh/Desktop/Macbook Working Files/Git Repos/650-DRAGON-SLAYERS/FILES/Josh-Dev/patient_df.csv')

patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5176 entries, 0 to 5175
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            5176 non-null   int64  
 1   SUBJECT_ID            5176 non-null   int64  
 2   GENDER                5176 non-null   object 
 3   DOB                   5176 non-null   object 
 4   DOD                   3278 non-null   object 
 5   HOSPITAL_EXPIRE_FLAG  5176 non-null   int64  
 6   HADM_ID               5176 non-null   int64  
 7   ADMITTIME             5176 non-null   object 
 8   DISCHTIME             5176 non-null   object 
 9   ADMISSION_TYPE        5176 non-null   object 
 10  ETHNICITY             5176 non-null   object 
 11  AGE_AT_ADMISSION      5176 non-null   int64  
 12  LOS                   5176 non-null   int64  
 13  LOS_ICU_MEAN          5175 non-null   float64
 14  WEIGHT_MEAN           4941 non-null   float64
 15  HEARTRATE_MEAN       